In [42]:
import pandas as pd
frame = None
file_path = "./simpsons_script_lines.csv"
frame = pd.read_csv(file_path, parse_dates = False, error_bad_lines = False, warn_bad_lines = False)

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [43]:
homer = frame[frame["character_id"] == 2]
homer_spoken_words = homer[~pd.isnull(homer["spoken_words"])]["spoken_words"]

In [44]:
homer_spoken_words

57        Never thrown a party? What about that big bash...
58        Bart didn't get one vote?! Oh, this is the wor...
62                                                      Oh.
64                                                     And?
66        Hey, just because I don't care doesn't mean I ...
68                                                      Me?
70            I don't think you realize what you're saying.
73        Did you hear that, Marge? She called me a babo...
75                                                 Go Away.
76        Lisa, don't hold anything back. You can tell m...
78                                                    Nuts.
79                            This isn't going well at all.
81              No, no, no. I just wish I knew what to say.
82        Although, maybe this will help. Now you lost s...
85                                                      Oh.
87        It's true. Now, you'll have lots of special pe...
89            I can hold these nails in 

In [45]:
from nltk import sent_tokenize
import unidecode
import re

preprocessed_sentences = []

def preprocess_sentence(sentence):
    sentence = unidecode.unidecode(sentence)
    sentence = re.sub(r"[-+]?(\d+([.,]\d*)?|[.,]\d+)([eE][-+]?\d+)?", r"#", sentence)
    sentence = re.sub(r"[\"-]+", r" ", sentence)
    sentence = re.sub(r"`", r"'", sentence)
    sentence = re.sub(r"[ ]+['][ ]+", r"'", sentence)
    sentence = re.sub(r"[ ]+['](?=[a-zA-Z ])", r" ", sentence)
    sentence = re.sub(r"(?<=[a-zA-Z])['][ ]+", r" ", sentence)
    sentence = re.sub(r"[\.]{2,}", r" ", sentence)
    sentence = re.sub(r"[$%&=|~<>/_\^\[\]{}():;,+*!?]+", r" ", sentence)
    sentence = re.sub(r"[ ]+", r" ", sentence)
    sentence = sentence.strip()
    sentence = re.sub(r"(?<=[a-zA-Z])[']$", r"", sentence)
    sentence = re.sub(r"^['](?=[a-zA-Z])", r"", sentence)
    sentence = re.sub(r"[\.][']$", r"", sentence)
    sentence = re.sub(r"['][\.]$", r"", sentence)
    sentence = re.sub(r"^[ ]", r"", sentence)
    sentence = re.sub(r"[ ]$", r"", sentence)
    sentence = re.sub(r"[\.]$", r"", sentence)
    sentence = sentence.strip()
    return sentence
            
def preprocess_and_append_line(line):
    sentences = sent_tokenize(line)
    for sentence in sentences:
        sentence = sentence.strip()
        sentence = preprocess_sentence(sentence)
        if (sentence != ""):
            preprocessed_sentences.append(sentence)
    
homer_spoken_words.apply(preprocess_and_append_line)

57        None
58        None
62        None
64        None
66        None
68        None
70        None
73        None
75        None
76        None
78        None
79        None
81        None
82        None
85        None
87        None
89        None
91        None
93        None
95        None
96        None
98        None
100       None
102       None
104       None
106       None
107       None
108       None
110       None
112       None
          ... 
130885    None
130898    None
130899    None
130901    None
130902    None
130903    None
130905    None
130947    None
130949    None
130951    None
130953    None
130955    None
130957    None
130988    None
130993    None
130995    None
131019    None
131021    None
131026    None
131027    None
131034    None
131046    None
131048    None
131050    None
131051    None
131052    None
131053    None
131055    None
131057    None
131067    None
Name: spoken_words, Length: 23073, dtype: object

In [46]:
chars = set()
for s in preprocessed_sentences:
    if ";" in s:
        print(s)
    for c in s:
        chars.add(c)

In [47]:
chars

{' ',
 '#',
 "'",
 '.',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z'}

In [48]:
preprocessed_sentences[:500]

['Never thrown a party',
 'What about that big bash we had with all the champagne and musicians and holy men and everything',
 "Bart didn't get one vote",
 'Oh this is the worst thing that ever happened to us',
 'Alright allright spilled milk spilled milk spilled milk',
 'What are you so mopey about',
 'Oh',
 'And',
 "Hey just because I don't care doesn't mean I don't understand",
 'Me',
 "I don't think you realize what you're saying",
 'Did you hear that Marge',
 'She called me a baboon',
 'The stupidest ugliest smelliest ape of them all',
 'Go Away',
 "Lisa don't hold anything back",
 'You can tell me',
 'Are you crying because you called Daddy a baboon',
 'Nuts',
 "This isn't going well at all",
 'No no no',
 'I just wish I knew what to say',
 'Although maybe this will help',
 'Now you lost someone special and it hurts',
 "I'm lucky because I never lost anyone special to me",
 'Everyone special to me is under this roof',
 'Oh',
 "It's true",
 "Now you'll have lots of special people 

In [49]:
from sklearn.model_selection import train_test_split
indices = range(len(preprocessed_sentences))
seed = 2092093
train, test, train_indices, test_indices = train_test_split(
    preprocessed_sentences,
    indices,
    train_size = 0.8,
    test_size = 0.2,
    random_state = seed)
test, valid, test_indices, valid_indices = train_test_split(
    test,
    test_indices,
    train_size = 0.5,
    test_size = 0.5,
    random_state = seed)

In [50]:
len(preprocessed_sentences)

38156

In [51]:
len(train_indices)

30524

In [52]:
len(test_indices)

3816

In [53]:
len(valid_indices)

3816

In [54]:
len(train_indices) / len(preprocessed_sentences)

0.7999790334416605

In [55]:
len(test_indices) / len(preprocessed_sentences)

0.10001048327916973

In [56]:
len(valid_indices) / len(preprocessed_sentences)

0.10001048327916973

In [57]:
def write_data(data, file_path):
    with open(file_path, "w") as file_out:
        for line in data:
            file_out.write(line + "\n")

write_data(train, "train.txt")
write_data(test, "test.txt")
write_data(valid, "valid.txt")